In [35]:
import tensorflow as tf
import numpy as np

In [36]:
english = [['I',' ','am',' ','a',' ','boy'], ['I',' ','am',' ','a',' ','girl'], ['How',' ','often',' ','do',' ','you',' ','eat',' ','per',' ','day','?'], ['You',' ','are',' ','so',' ','beautiful'], ['Would', ' ', 'you', ' ', 'mind', ' ', 'helping', ' ', 'me', ' ', 'out', '?'], ['deep', ' ', 'learning', ' ', 'is', ' ', 'really', ' ', 'funny', '.']]
korean = [['나','는', ' ', '소년','이다', '.'], ['나','는', ' ', '소녀', '다', '.'], ['하루에', ' ', '얼마나', ' ', '자주', ' ', '드십니까','?'], ['당신','은', ' ', '너무', ' ', '아름답다', '.'], ['혹시', ' ', '도와', ' ', '주실', ' ', '수', ' ', '있나요', '?'], ['딥러닝', ' ', '진짜', ' ', '재밌네', '.']]
seq_data = np.stack((english, korean), 1)

In [39]:
seq_data

array([[list(['I', ' ', 'am', ' ', 'a', ' ', 'boy']),
        list(['나', '는', ' ', '소년', '이다', '.'])],
       [list(['I', ' ', 'am', ' ', 'a', ' ', 'girl']),
        list(['나', '는', ' ', '소녀', '다', '.'])],
       [list(['How', ' ', 'often', ' ', 'do', ' ', 'you', ' ', 'eat', ' ', 'per', ' ', 'day', '?']),
        list(['하루에', ' ', '얼마나', ' ', '자주', ' ', '드십니까', '?'])],
       [list(['You', ' ', 'are', ' ', 'so', ' ', 'beautiful']),
        list(['당신', '은', ' ', '너무', ' ', '아름답다', '.'])],
       [list(['Would', ' ', 'you', ' ', 'mind', ' ', 'helping', ' ', 'me', ' ', 'out', '?']),
        list(['혹시', ' ', '도와', ' ', '주실', ' ', '수', ' ', '있나요', '?'])],
       [list(['deep', ' ', 'learning', ' ', 'is', ' ', 'really', ' ', 'funny', '.']),
        list(['딥러닝', ' ', '진짜', ' ', '재밌네', '.'])]], dtype=object)

In [37]:
word_arr = []
for sents in seq_data:
    for seq in sents:
        word_arr += seq
word_arr += ['<S>', '</S>', '<P>']
word2num = {c:i for i, c in enumerate(set(word_arr))}
num2word = {i:c for i, c in enumerate(word2num.keys())}
dic_len = len(word2num)

In [21]:
def get_max_length(seq_data):
    max_len = 0
    for seq in seq_data:
        if max_len < len(seq):
            max_len = len(seq)
    return max_len

In [22]:
def make_batch(seq_data, enc_max_len, dec_max_len):
    input_batch = []
    output_batch = []
    target_batch = []
    for i, seq in enumerate(seq_data):
        input = []
        output = []
        target = []
        for token in seq[0]:
            input.append(word2num[token])
        for _ in range(len(seq[0]), enc_max_len):
            input.append(word2num['<P>'])
        input_batch.append(np.eye(dic_len)[input])
        output.append(word2num['<S>'])
        for token in seq[1]:
            output.append(word2num[token])
            target.append(word2num[token])
        target.append(word2num['</S>'])
        for _ in range(len(seq[1]), dec_max_len):
            output.append(word2num['</S>'])
            target.append(word2num['</S>'])
                
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [23]:
def get_seq_length(seq_data):
    seq_len = []
    for i, seq in enumerate(seq_data):
        seq_len.append(len(seq))
    return seq_len

In [24]:
learning_rate = 1e-3
n_hidden = 256
max_enc_step = get_max_length(english)
max_dec_step = get_max_length(korean)
total_epoch = 100
n_class = n_input = dic_len
batch_size = len(english)

In [25]:
tf.reset_default_graph()
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])
enc_seq_len = tf.placeholder(dtype=tf.int32, shape=[None])
dec_seq_len = tf.placeholder(dtype=tf.int32, shape=[None])

In [26]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    #enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, sequence_length=enc_seq_len, dtype=tf.float32)

In [27]:
#idx = tf.range(tf.shape(outputs)[0])*tf.shape(outputs)[1] + (enc_seq_len - 1)
#idx = tf.cast(tf.reduce_sum(tf.one_hot(idx, tf.shape(outputs)[0]*tf.shape(outputs)[1]), 0), tf.int32)
#outputs = tf.dynamic_partition(tf.reshape(outputs, [-1, n_hidden]), idx, 2)
#last_enc_states = outputs[1]

In [28]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    #dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state = enc_states, dtype=tf.float32)

In [29]:
logits = tf.layers.dense(outputs, n_class, activation=None)

In [30]:
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=logits, targets=targets, weights=tf.sequence_mask(dec_seq_len+1, max_dec_step+1, dtype=tf.float32)))
predict = tf.argmax(logits, 2)

In [31]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [32]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data, max_enc_step, max_dec_step)
enc_seq_data = get_seq_length(english)
dec_seq_data = get_seq_length(korean)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch, enc_seq_len: enc_seq_data, dec_seq_len: dec_seq_data})
    if epoch % 10 == 9:
        result = sess.run(predict, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch, enc_seq_len: enc_seq_data, dec_seq_len: dec_seq_data})
        translation = []
        for i in range(len(result)):
            decoded = [num2word[j] for j in result[i]]
            end = len(decoded)-1
            if '</S>' in decoded:
                end = decoded.index('</S>')
            translated = ''.join(decoded[:end])
            translation.append(translated)
        print('Epoch:', '%03d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
        print(translation)
print('optimization finished!')

Epoch: 010 cost = 1.077412
['나는 소녀이다.', '나는 소녀이다.', '하루에 얼마나 주실 ', '나는 너무 ..', '혹시 도와 주실 ', '하루에 진짜 주실.']
Epoch: 020 cost = 0.268277
['나는 소년이다.', '나는 소녀다.', '하루에 얼마나 자주 드십니까?', '당신은 너무 아름답다.', '혹시 도와 주실 수 있나요?', '딥러닝 진짜 재밌네.']
Epoch: 030 cost = 0.092709
['나는 소년이다.', '나는 소년다.', '하루에 얼마나 자주 드십니까?', '당신은 너무 아름답다.', '혹시 도와 주실 수 있나요?', '딥러닝 진짜 재밌네.']
Epoch: 040 cost = 0.061266
['나는 소년이다.', '나는 소녀다.', '하루에 얼마나 자주 드십니까?', '당신은 너무 아름답다.', '혹시 도와 주실 수 있나요?', '딥러닝 진짜 재밌네.']
Epoch: 050 cost = 0.049847
['나는 소년이다.', '나는 소녀다.', '하루에 얼마나 자주 드십니까?', '당신은 너무 아름답다.', '혹시 도와 주실 수 있나요?', '딥러닝 진짜 재밌네.']
Epoch: 060 cost = 0.040163
['나는 소년이다.', '나는 소녀다.', '하루에 얼마나 자주 드십니까?', '당신은 너무 아름답다.', '혹시 도와 주실 수 있나요?', '딥러닝 진짜 재밌네.']
Epoch: 070 cost = 0.026008
['나는 소년이다.', '나는 소녀다.', '하루에 얼마나 자주 드십니까?', '당신은 너무 아름답다.', '혹시 도와 주실 수 있나요?', '딥러닝 진짜 재밌네.']
Epoch: 080 cost = 0.011238
['나는 소년이다.', '나는 소녀다.', '하루에 얼마나 자주 드십니까?', '당신은 너무 아름답다.', '혹시 도와 주실 수 있나요?', '딥러닝 진짜 재밌네.']
Epoch: 090 cost = 0.006324
['나는 소년이다.', '나는 소녀다

In [33]:
def translate(english):
    english = [english]
    korean = [['<P>']*max_dec_step]
    seq_data = [english + korean]
    input_batch, output_batch, target_batch = make_batch(seq_data, max_enc_step, max_dec_step)
    enc_seq_data = get_seq_length(english)
    dec_seq_data = get_seq_length(korean)
    result = sess.run(predict, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch, enc_seq_len: enc_seq_data, dec_seq_len: dec_seq_data})
    decoded = [num2word[j] for j in np.squeeze(result)]
    end = len(decoded)-1
    if '</S>' in decoded:
        end = decoded.index('</S>')
    translated = ''.join(decoded[:end])
    return translated

In [34]:
for seq in english:
    temp = ''
    for token in seq:
        temp += token
    #print(temp)
    print(temp, '->', translate(seq))

I am a boy -> 나는 소년이다.
I am a girl -> 나는 소녀다.
How often do you eat per day? -> 하루에 얼마나 자주 드십니까?
You are so beautiful -> 당신은 너무 아름답다.
Would you mind helping me out? -> 혹시 도와 주실 수 있나요?
deep learning is really funny. -> 딥러닝 진짜 재밌네.
